In [11]:
def two_opt (dist_mat):
    
    max_iter = 10**5
    X_0 = 0.8
    alpha = 1 - 1/max_iter**X_0 # geometric scalar to lower temp slowly

    init_path = SA_TS(dist_mat.to_numpy(), max_iter = max_iter, alpha = alpha, X_0 = X_0, reheat_threshold = max_iter/5)[:-1]

    def cost_func(state):
        cost = 0 
        for i in range(len(state) - 1):
            cost += dist_mat.loc[state[i], state[i+1]]
        cost += dist_mat.loc[state[-1],state[0]]  
        if len(state) == len(dist_mat):
            cost += dist_mat.loc[state[-1], 0]      
        return cost
    
    def switch_two(state, a, b):
        a_ind = state.index(a)
        b_ind = state.index(b)
        state[a_ind], state[b_ind] = state[b_ind], state[a_ind]
        return state 
    
    new_state = init_path
    un_swapped = [i for i in new_state]
    current_cost = cost_func(new_state)
    
    for j in un_swapped:
        
        state = new_state
        cur = un_swapped[j]
        temp = [k for k in un_swapped if k != j]
        
        for i in temp:
        
            opt = switch_two(state, cur, i)    
            opt_cost = cost_func(opt)
            
            if opt_cost < current_cost:
                current_cost = opt_cost
                new_state = opt
                
    state.append(state[0])  
    
    return state,current_cost

In [13]:
import numpy as np
import pandas as pd
from python_tsp.exact import solve_tsp_dynamic_programming
from python_tsp.heuristics import solve_tsp_local_search

def write_distance_matrix(n, mean, sigma):
    distance_matrix = np.zeros((n, n))
    random_distance = []
    num_distance = int(n * (n-1) / 2)
    for _ in range(num_distance):
        distance = 0
        while distance <= 0:
            distance = np.random.normal(mean, sigma)

        random_distance.append(round(distance))
    
    iu = np.triu_indices(n, 1)
    distance_matrix[iu] = random_distance
    distance_matrix += distance_matrix.T

    return distance_matrix

N = 20
dist_mat = pd.DataFrame(write_distance_matrix(N, 10, 100))
state = [i for i in range(N)]

permutation, distance = solve_tsp_dynamic_programming(dist_mat.to_numpy())

p, d = two_opt(dist_mat)

print('dist', distance)
print('2opt', d)

dist 149.0
2opt 1126.0


In [12]:
import numpy as np
import random 
import pandas as pd
import matplotlib.pyplot as plt

def SA_TS(dist_mat, max_iter, alpha, X_0, reheat_threshold):
    
    # dist_mat: nxn symmetric numpy array 
    # max_iter: stopping criteria
    # alpha: geometric temperature scalar - a function of max_iter, current iteration, and desired acceptance prob X_0
    # X_0: initial desired acceptance
    
    p_array = []
    distance_array = []
    temp_array = []
    swap_types = ['Swap 2', 'Insert', 'Swap Subroute', 'Invert']
    
    # Total distance for current path
    def objective_function(state):
        cost = 0
        for i in range(len(state) - 1):
            cost += dist_mat[state[i], state[i+1]]
        cost += dist_mat[state[-1],state[0]] 
        return cost
    
    
    # construct initial path
    def init_path():
        # Initial path: 0 -> 1, 1 -> 2, ... N-1 -> N -> 0
        initial = [i for i in range(len(dist_mat))]
        return initial
    
    # Computing the Initial Temperature of Simulated Annealing, Walid 2003
    # idea: sample SS bad transitions and calculate the acceptance ratio for this sample: iteratively reduce T until desired prob X_0
    def init_temp(X_0):
        
        
        s_trials = 0
        E_after = []
        E_before = []
        
        SS = int(max_iter/3) # trains such that we get X_0 acceptance for the first 10% of runtime
        
        # generate SS random positive transitions
        current_sample_path = init_path()
        current_sample_distance = objective_function(current_sample_path)
        
        while s_trials < SS:
            new_sample_path = choose_apply_action(current_sample_path)
            new_sample_distance = objective_function(new_sample_path)
            E = current_sample_distance - new_sample_distance 

            if E < 0:
                E_before.append(current_sample_distance) #Emin
                E_after.append(new_sample_distance) #Emax
                s_trials += 1
                
                if np.random.binomial(1, X_0) == 1:
                    current_sample_distance = new_sample_distance
                    current_sample_path = new_sample_path
            else:
                current_sample_path = new_sample_path
                current_sample_distance = new_sample_distance
        
        # Iteratively adjust initial temperature until acceptance ratio (X_Tn) is approximately equal X_0
        start = True
        X_Tn = 0
        Tn = 10*len(dist_mat) # initial starting temperature (only need Tn > 1)
        
        while abs(X_Tn - X_0) > 0.01:     
            
            num = 0
            denom = 0
            # without this check we terminate on the first iteration
            if start == True:
                X_Tn = X_0
                start = False
                
            Tn = abs(Tn*(np.log(X_Tn)/np.log(X_0)))
            
            for i in range(SS):
                num += np.exp(-E_after[i]/Tn)
                denom += np.exp(-E_before[i]/Tn)
                
            X_Tn = num/denom
            
        return Tn
    
    # path changing actions (4)
    def switch_two(state):
        # ex: a,b = 2,5
        # [0, 1, 2, 3, 4, 5, 6, 0] -> [0, 1, 5, 3, 4, 2, 6,0]
        a = random.randint(0, len(state)-1)
        b = random.randint(0, len(state)-1)
        while b == a:
            b = random.randint(0,len(state)-1)
        state[a], state[b] = state[b], state[a]
        return state
    
    def invert_path_between(state):
        # ex: a,b = 2,5
        # [0, 1, 2, 3, 4, 5, 6, 0] -> [0, 1, 4, 3, 2, 5, 6, 0]
        a = random.randint(0,len(state)-1)
        b = random.randint(0,len(state)-1)
        while b == a:
            b = random.randint(0,len(state)-1)    
        if b < a:
            a,b = b,a   
        state[a:b] = state[a:b][::-1] # reverse sublist
        return state

    def insert_random(state):
        # ex: a,b = 2,5
        # [0, 1, 2, 3, 4, 5, 6, 0] -> [0, 1, 3, 4, 5, 2, 6, 0]
        a = random.randint(0,len(state)-1)
        b = random.randint(0,len(state)-1)
        while b == a:
            b = random.randint(0,len(state)-1)    
        city = state.pop(a)
        state.insert(b, city) 
        return state
    
    def swap_subroute(state):
        # move a whole subroute
        a = random.randint(0,len(state)-1)
        b = random.randint(0,len(state)-1)
        while b == a:
            b = random.randint(0, len(state)-1)  
        if b < a:
            a,b = b,a  
        subroute = state[a:b]
        state = [state[x] for x in range(len(state)) if state[x] not in subroute]
        c = random.randint(0, len(state))
        state[c:c] = subroute
        return state
    
    # randomly choose and apply an action
    def choose_apply_action(path):
        
        random_type = random.choice(swap_types)

        if random_type == 'Swap 2':
            new_path = switch_two(path)
        elif random_type == 'Insert':
            new_path = insert_random(path)
        elif random_type == 'Swap Subroute':
            new_path = swap_subroute(path)
        elif random_type == 'Invert':
            new_path = invert_path_between(path)
        return new_path
    
    # first path and initializations
    
    T = init_temp(X_0)
    
    heat = 0
    stagnation = 0
    reheat = 1
    
    path = init_path()
    current_distance = objective_function(path)
    best_distance = current_distance
    best_path = path
    
    # begin algorithm            
    for t in range(max_iter):
        distance_array.append(best_distance) # for plot
        
        # find new path and path's distance
        new_path = choose_apply_action(path)
        new_distance = objective_function(new_path)
        E = current_distance - new_distance
            
        # always track best path     
        if new_distance < best_distance:
            best_distance = new_distance
            best_path = new_path
            best_path.append(best_path[0])
            
        heat += 1
        
        # new path is shorter
        if E > 0:
            current_distance = new_distance
            path = new_path
            p_array.append(0)
        
        else:
                            
            # temperature function and probability (most sensitive part of the algorithm)   
            temp = T*alpha**heat
            p = np.exp(E/temp)/(reheat**0.8) # penalize reheating
            
            # roll whether to keep longer new path anyway
            if np.random.binomial(1, p) == 1:
                current_distance = new_distance
                path = new_path
                stagnation = 0 
                
            # increase stagnation factor: the longer we go with no bad rolls the more stagnated we become
            else:
                
                stagnation += 1
                
                # reset temperature
                if stagnation >= reheat_threshold:
                    
                    heat = 100*reheat
                    reheat_threshold *= 1.1 # increase threshold such that reheats become rarer each time
                    stagnation = 0
                    reheat += 1
                    
            temp_array.append(temp)
            p_array.append(p)
            

    return best_path

In [10]:
A = [0,1,2,3]
A[:-1]

[0, 1, 2]